# Evaluation of A **Regression** Model with Seperate Classes

This notebook is intended to be used for evaluation of a trained regression model over test dataset. It is the upper version of the regression model which evaluates regression of each classes seperatelly. Please change the config dictionary in the begening so that you can evaluate your desired model/sample etc...

In [ ]:
#configure your evalutaion parameters here
_config = {
    'batch_size':1,
    'input_shape_height' : 375,
    'input_shape_width' : 262,
    'model_filename' : 'output/resnet_regrerssion_classed/resnet_v1.h5',
    'test_sample_filename' : 'data/test/00000000a9764df2_20191005_085442_4.png'
}

In [ ]:
#If you want to disable GPU uncomment here
#import os 
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [ ]:
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.applications.resnet_v2 import preprocess_input

physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    _ = tf.config.experimental.set_memory_growth(physical_devices[0], True)

df_train =  pd.read_csv('data/train_labels.csv')
df_test =  pd.read_csv('data/test_labels.csv')

from helpers.decouple import decouple
matrix_test,classes_test = decouple(df_test)
from helpers.matrix_to_df import matrix_to_dfclassed
df_test_agg = matrix_to_dfclassed(matrix_test,classes_test)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = test_datagen.flow_from_dataframe(
        dataframe=df_test_agg,
        directory='data/test',
        x_col='filename',
        y_col=list(classes_test.keys()),
        target_size=(_config['input_shape_height'], _config['input_shape_width']),
        batch_size=_config['batch_size'],
        class_mode='raw')

model = load_model(_config['model_filename'],compile=False)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

In [ ]:
model.metrics_names

In [ ]:
model.evaluate_generator(test_generator)